# This notebook is prepared by Vasilii Sitdikov as a solution for assignment

### Week 1
### Course: Pattern Discovery in Data Mining
### Specialization: Data Mining
###  University of Illinois MCS in Data Science (Coursera)

## Description
##### In this programming assignment, you are required to implement the Apriori algorithm and apply it to mine frequent itemsets from a real-life data set.

### Input
The provided input file ("categories.txt") consists of the category lists of 77,185 places in the US. Each line corresponds to the category list of one place, where the list consists of a number of category instances (e.g., hotels, restaurants, etc.) that are separated by semicolons.
An example line is provided below:

#### Local Services;IT Services & Computer Repair

In the example above, the corresponding place has two category instances: "Local Services" and "IT Services & Computer Repair".

### Output
You need to implement the Apriori algorithm and use it to mine category sets that are frequent in the input data. When implementing the Apriori algorithm, you may use any programming language you like. We only need your result pattern file, not your source code file.

After implementing the Apriori algorithm, please set the relative minimum support to 0.01 and run it on the 77,185 category lists. In other words, you need to extract all the category sets that have an absolute support larger than 771.

#### Part 1

Please output all the length-1 frequent categories with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one frequent category and should be in the following format:

#### support:category

For example, suppose a category (Fast Food) has an absolute support 3000, then the line corresponding to this frequent category set in "patterns.txt" should be:

#### 3000:Fast Food

#### Part 2

Please write all the frequent category sets along with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one frequent category set and should be in the following format:

#### support:category_1;category_2;category_3;...

For example, suppose a category set (Fast Food; Restaurants) has an absolute support 2851, then the line corresponding to this frequent category set in "patterns.txt" should be:

#### 2851:Fast Food;Restaurants

### Important Tips
Make sure that you format each line correctly in the output file. For instance, use a semicolon instead of another character to separate the categories for each frequent category set.

In the result pattern file, the order of the categories does not matter. For example, the following two cases will be considered equivalent by the grader:

Case 1:

2851:Fast Food;Restaurants

Case 2:

2851:Restaurants;Fast Food

# Solution

## I. Loading and parsing the file

#### Install library for work with enviroment (we need to get the txt file)

In [269]:
!pip install ibm-cos-sdk

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


#### Getting credentials

In [270]:
# The code was removed by Watson Studio for sharing.

#### Import libraries and define functions for getting the text from enviroment

In [271]:
import ibm_boto3
from botocore.client import Config

In [272]:
cos = ibm_boto3.client('s3',
                       ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                       ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                       ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                       config=Config(signature_version='oauth'),
                       endpoint_url=credentials_1['ENDPOINT'])

In [273]:
def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=filename)['Body']
    return fileobject

In [274]:
def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    return text

#### Getting the file as a stream object and decoding to the string

In [275]:
file = get_file('Pattern Assignment .txt')
text = load_string(file).decode()
print(text)

Breakfast & Brunch;American (Traditional);Restaurants
Sandwiches;Restaurants
Local Services;IT Services & Computer Repair
Restaurants;Italian
Food;Coffee & Tea
Fast Food;Restaurants
Mortgage Brokers;Home Services;Real Estate
Brasseries;Restaurants
Bars;Sports Bars;Nightlife;American (New);Chicken Wings;Restaurants
Automotive;Windshield Installation & Repair;Auto Detailing;Wheel & Rim Repair
Automotive;Auto Parts & Supplies
Food;Grocery;CSA;Farmers Market
Specialty Schools;CPR Classes;First Aid Classes;Education
Event Planning & Services;Venues & Event Spaces
Shopping;Home Decor;Home & Garden;Furniture Stores
Books, Mags, Music & Video;Shopping;Bookstores
Auto Repair;Automotive
Local Services;Dry Cleaning & Laundry
Burgers;American (New);Restaurants
Pizza;Restaurants
Massage;Beauty & Spas
Food;Juice Bars & Smoothies
Pizza;Restaurants
Bars;Nightlife;Lounges
Champagne Bars;Bars;Nightlife;Lounges
Burgers;Restaurants
Bars;American (Traditional);Nightlife;Restaurants
Photographers;Session Ph

#### Parsing the text file to the 2D list "data" 

In [276]:
data=[]
for num, line in enumerate(text.splitlines()):
    data.append(str(line).split(';'))
data[0:5][:]

[['Breakfast & Brunch', 'American (Traditional)', 'Restaurants'],
 ['Sandwiches', 'Restaurants'],
 ['Local Services', 'IT Services & Computer Repair'],
 ['Restaurants', 'Italian'],
 ['Food', 'Coffee & Tea']]

 ## II. Create the first (one-word) dictionary of friquency

#### Create a funcion to return the dictionary of friquency of elements in the 2D list "data" if friquency is greater then "threshold"

In [277]:
def f_count(data, threshold):
    """ Return the dictionary of friquency of elements in the 2D list "data" if friquency is greater then "threshold" """ 
    dict = {}
    for row in data:
        for item in row:
            if item in dict.keys():
                dict[item] = dict[item] + 1
            else:
                dict[item] = 1
    dict2 = {}
    for key, val in dict.items():
        if val >= threshold:
            keyt = tuple([key])
            dict2[keyt] = val
    return dict2

#### Create the first data for the output

In [278]:
#Difine threshold (from the task's conditions)
threshold = 771

#Create the list "output" and add one-word dictionary for output to the first (0) place of list
output = []
output.append(f_count(data, threshold))

print(output)

[{('Auto Repair',): 1716, ('Fitness & Instruction',): 1442, ('Beauty & Spas',): 6583, ('Real Estate',): 1424, ('Burgers',): 1774, ('Financial Services',): 875, ('Chinese',): 1629, ('Shopping',): 11233, ('Mexican',): 2515, ('Fashion',): 3078, ('Ice Cream & Frozen Yogurt',): 1018, ('Local Services',): 3468, ('Italian',): 1848, ("Women's Clothing",): 1138, ('Sushi Bars',): 798, ('Japanese',): 848, ('Bakeries',): 1115, ('Grocery',): 1424, ('Home Services',): 4785, ('Arts & Entertainment',): 2271, ('Hotels & Travel',): 2495, ('Food',): 9250, ('Nightlife',): 5088, ('Pet Services',): 870, ('Doctors',): 1694, ('General Dentistry',): 823, ('Dentists',): 1195, ('Sports Bars',): 818, ('Bars',): 4328, ('American (New)',): 1593, ('Health & Medical',): 5121, ('Fast Food',): 2851, ('Pizza',): 2657, ('Cafes',): 1002, ('Breakfast & Brunch',): 1369, ('Pubs',): 874, ('Specialty Food',): 1150, ('Pets',): 1497, ('Coffee & Tea',): 2199, ('Active Life',): 3103, ('Professional Services',): 1025, ('Nail Salons

## III. Create the other dictionaries of friquency

### Neccessary functions

#### A function to extract keys of dictionary to list

In [279]:
def extract_keys_to_list(dict):
    """Extract keys of dictionary to list"""
    list = []
    for key in dict:
        list.append(key)
    return list

#### A function to create combinations of tuple and words 

In [280]:
def combination_n(list_n, list_1):
    """Return combinations of words in tuples of list_n and words in one-word list_1"""
    list = []
    for item_n in list_n:
        for item_1 in list_1:
            if item_1[0] > item_n[-1]:
                list.append((item_n + item_1))
    return list

#### A function to count frequency of tuple-sets in rows of 2D list

In [281]:
def fn_count(data = [], dict_in = [()], threshold = 0):
    """ Return the dictionary of friquency of elements (checced sets) in the 2D list "data" if friquency is greater then "threshold"
        This function gets as input a dictionary of tuples, every tuple is the schecked set""" 
    dict_out = {}
    # Loop through input (checked) list of tuples (sets)
    for check_set in dict_in:
        # Loop through data list (rows)
        for row in data:
            result = True
            #Loop through check_set (every items in the tuple)
            for item in check_set:
                result = item in row
                if not result: break
            if result:
                if check_set in dict_out.keys():
                    dict_out[check_set] += 1
                else:
                    dict_out[check_set] = 1
    dict2 = {}
    global added
    for key, val in dict_out.items():
        if val >= threshold:
            dict2[key] = val
            added = True
    return dict2    

## The main executive loop

In [282]:
added = True      #The global variable to be sure that new dictionary isn't empty. Used as global in fn_count
one_word_keys = sorted(extract_keys_to_list(output[0])) #list of tuples (one word).
i=1
while added:
    added = False
    derived_keys = sorted(extract_keys_to_list(output[-1])) #list of tuples. Derive keys from the previous step
    new_combinations = combination_n(derived_keys, one_word_keys) #List of tuples. Combinations of derived keys and one-word list
    output.append(fn_count(data, new_combinations, threshold)) #List of outputs
    print(i)
    i += 1
print(output)

1
2
3
[{('Auto Repair',): 1716, ('Fitness & Instruction',): 1442, ('Beauty & Spas',): 6583, ('Real Estate',): 1424, ('Burgers',): 1774, ('Financial Services',): 875, ('Chinese',): 1629, ('Shopping',): 11233, ('Mexican',): 2515, ('Fashion',): 3078, ('Ice Cream & Frozen Yogurt',): 1018, ('Local Services',): 3468, ('Italian',): 1848, ("Women's Clothing",): 1138, ('Sushi Bars',): 798, ('Japanese',): 848, ('Bakeries',): 1115, ('Grocery',): 1424, ('Home Services',): 4785, ('Arts & Entertainment',): 2271, ('Hotels & Travel',): 2495, ('Food',): 9250, ('Nightlife',): 5088, ('Pet Services',): 870, ('Doctors',): 1694, ('General Dentistry',): 823, ('Dentists',): 1195, ('Sports Bars',): 818, ('Bars',): 4328, ('American (New)',): 1593, ('Health & Medical',): 5121, ('Fast Food',): 2851, ('Pizza',): 2657, ('Cafes',): 1002, ('Breakfast & Brunch',): 1369, ('Pubs',): 874, ('Specialty Food',): 1150, ('Pets',): 1497, ('Coffee & Tea',): 2199, ('Active Life',): 3103, ('Professional Services',): 1025, ('Nail 

## IV. Write output to file

#### Create the loop to write down output to the txt file in required format

In [283]:
with open('patterns.txt', 'w') as file:
    # Loop in output by "dimension" - dictionaries with sets of words 
    for dimension in output:
        #Loop in dimensions dictionary to derive sets of words
        for sets_key in dimension.keys():
            string = str(dimension[sets_key])+':'
            for value in sets_key:
                string += value + ';'
            file.write(string[:-1]+'\r\n')

In [284]:
with open('patterns.txt', 'r') as file:
    print(file.read())

1716:Auto Repair
1442:Fitness & Instruction
6583:Beauty & Spas
1424:Real Estate
1774:Burgers
875:Financial Services
1629:Chinese
11233:Shopping
2515:Mexican
3078:Fashion
1018:Ice Cream & Frozen Yogurt
3468:Local Services
1848:Italian
1138:Women's Clothing
798:Sushi Bars
848:Japanese
1115:Bakeries
1424:Grocery
4785:Home Services
2271:Arts & Entertainment
2495:Hotels & Travel
9250:Food
5088:Nightlife
870:Pet Services
1694:Doctors
823:General Dentistry
1195:Dentists
818:Sports Bars
4328:Bars
1593:American (New)
5121:Health & Medical
2851:Fast Food
2657:Pizza
1002:Cafes
1369:Breakfast & Brunch
874:Pubs
1150:Specialty Food
1497:Pets
2199:Coffee & Tea
3103:Active Life
1025:Professional Services
1667:Nail Salons
4208:Automotive
25071:Restaurants
2364:Sandwiches
2975:Event Planning & Services
1431:Hotels
2091:Hair Salons
1586:Home & Garden
2416:American (Traditional)
1431:Event Planning & Services;Hotels
870:Pet Services;Pets
2515:Mexican;Restaurants
2533:Nightlife;Restaurants
2199:Coffee & Te

#### Upload file to the Cloud Object Storage

In [287]:
resource = ibm_boto3.resource('s3',
                   ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                   ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                   ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                   config=Config(signature_version='oauth'),
                   endpoint_url=credentials_1['ENDPOINT'])
resource.Bucket(name=credentials_1['BUCKET']).put_object(Key='patterns.txt', Body=open('patterns.txt', 'rb'))

s3.Object(bucket_name='assignmentfrequentitemsetminingus-donotdelete-pr-bsxgb7luqmjbzs', key='patterns.txt')